<a href="https://colab.research.google.com/github/theekshaya4/Image-colorization/blob/main/task_theekynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision opencv-python pillow streamlit kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 53.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: cachetools
    Found existing installation: cachetools 7.0.0
    Uninstalling cachetools-7.0.0:
      Successfully uninstalled cachetools-7.0.0


In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d shravankumar9892/image-colorization
!unzip image-colorization.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'
unzip:  cannot find or open image-colorization.zip, image-colorization.zip.zip or image-colorization.zip.ZIP.


In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
from google.colab import files # Added for file upload

# --- Start of added code to handle Kaggle setup and data download ---
# Check if kaggle.json exists, if not, prompt for upload
kaggle_json_path = "kaggle.json"
if not os.path.exists(kaggle_json_path):
    print("kaggle.json not found. Please upload your kaggle.json file.")
    uploaded = files.upload()
    if kaggle_json_path not in uploaded:
        raise FileNotFoundError(f"{kaggle_json_path} was not uploaded. Cannot proceed with Kaggle dataset download.")

# Configure Kaggle API
kaggle_dir = os.path.expanduser('~/.kaggle')
os.makedirs(kaggle_dir, exist_ok=True)
!cp {kaggle_json_path} {kaggle_dir}/
!chmod 600 {kaggle_dir}/kaggle.json
kaggle_dataset_slug = "shravankumar9892/image-colorization"
dataset_zip_file = "image-colorization.zip"
dataset_extract_dir = "/content/images" # This is the target path for the dataset

# Check if dataset is already extracted
if not os.path.exists(dataset_extract_dir) or not os.listdir(dataset_extract_dir):
    print(f"Dataset not found or empty in {dataset_extract_dir}. Downloading and extracting...")
    # Download dataset
    !kaggle datasets download -d {kaggle_dataset_slug} -p /content/
    if os.path.exists(f"/content/{dataset_zip_file}"):
        os.makedirs(dataset_extract_dir, exist_ok=True) # Ensure target dir exists for unzip
        !unzip -q /content/{dataset_zip_file} -d {dataset_extract_dir}
        print(f"Dataset extracted to {dataset_extract_dir}")
    else:
        raise FileNotFoundError(f"Kaggle dataset '{dataset_zip_file}' not found after download attempt.")
else:
    print(f"Dataset already present in {dataset_extract_dir}. Skipping download and extraction.")

# --- End of added code ---
# ---------------- DEVICE ----------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

# ---------------- DATASET ----------------
class ColorDataset(Dataset):
    def __init__(self, path):
        self.path = path

        # Filter only image files
        self.files = [f for f in os.listdir(path)
                      if f.lower().endswith((".png", ".jpg", ".jpeg"))]

        if len(self.files) == 0:
            raise ValueError("No images found in dataset folder!")

        self.transform = transforms.Compose([
            transforms.Resize((128, 128)),
            transforms.ToTensor()
        ])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.path, self.files[idx])

        try:
          img = Image.open(img_path).convert("RGB")
        except:
            # Skip corrupted image
            return self.__getitem__((idx + 1) % len(self.files))

        gray = img.convert("L")

        gray_tensor = self.transform(gray)
        color_tensor = self.transform(img)

        return gray_tensor, color_tensor


# 🔴 CHANGE THIS PATH IF NEEDED
dataset_path = "/content/images"

dataset = ColorDataset(dataset_path)
loader = DataLoader(dataset, batch_size=16, shuffle=True)

print("Dataset Loaded:", len(dataset), "images")

# ---------------- MODEL ----------------
class ColorNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv2d(1, 64, 3, padding=1),
            nn.ReLU(),

            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(),

            nn.Conv2d(128, 64, 3, padding=1),
            nn.ReLU(),

            nn.Conv2d(64, 3, 3, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.next(x)
model = ColorNet().to(device)

# ---------------- TRAIN SETUP ----------------
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ---------------- TRAINING ----------------
epochs = 3

for epoch in range(epochs):

    total_loss = 0

    for gray, color in loader:
        gray = gray.to(device)
        color = color.to(device)

        output = model(gray)

        loss = loss_fn(output, color)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)

    print(f"Epoch {epoch+1}/{epochs}  |  Loss: {avg_loss:.4f}")

kaggle.json not found. Please upload your kaggle.json file.
